In [ ]:
import os
import json
import boto3
import sagemaker
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth, helpers
import s3fs

In [ ]:
with open("oss_policies_info.json", "r") as f:
    saved_data = json.load(f)

host = saved_data["opensearch_host"]
index_name = saved_data["opensearch_index_name"]
vector_store_name = saved_data["vector_store_name"]
encryption_policy = saved_data["encryption_policy"]
network_policy = saved_data["network_policy"]
access_policy = saved_data["access_policy"]


print(f"""OpenSearch Host: {host}\n \
        Index Name    : {index_name}""")

print(f"""Vector Store Name: {vector_store_name}\n \
        Encryption Policy: {encryption_policy}\n \
        Network Policy   : {network_policy} \n \
        Access Policy    : {access_policy}""")

In [ ]:
## Initialize boto3 session ## 
boto3_session = boto3.session.Session(region_name='us-west-2')
print(f'The notebook will use aws services hosted in {boto3_session.region_name} region')

# initialize boto3 clients for required AWS services
sts_client = boto3_session.client('sts')
s3_client = boto3_session.client('s3')
opensearchservice_client = boto3_session.client('opensearchserverless')

service = 'aoss'
credentials = boto3.Session().get_credentials()
awsauth = AWSV4SignerAuth(credentials, boto3_session.region_name, service)

oss_client = OpenSearch(
    hosts=[{'host': host, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=600
)

In [ ]:
## clean up 
# delete vector index
try:
    oss_client.indices.delete(index=index_name)
    print('Index Deleted')
except Exception as e:
    print('Index already deleted or not found')

# delete data, network, and encryption access ploicies
try:
    opensearchservice_client.delete_access_policy(type="data", name=access_policy['accessPolicyDetail']['name'])
    print('Access Policy Deleted')
    opensearchservice_client.delete_security_policy(type="network", name=network_policy['securityPolicyDetail']['name'])
    print('Network Security Policy Deleted')
    opensearchservice_client.delete_security_policy(type="encryption", name=encryption_policy['securityPolicyDetail']['name'])
    print('Encryption Security Policy Deleted')
except Exception as e:
    print('Access and Security Policies already deleted or not found')

# delete collection
try:
    response = opensearchservice_client.list_collections()
    for collection in response['collectionSummaries']:
        if collection['name'] == vector_store_name:
            vs_collection_id = collection.get('id', 'unknown')
            opensearchservice_client.delete_collection(id=vs_collection_id)
            print('OpenSearch Collection Deleted')
except Exception as e:
    print('Collection already deleted or not found')